# Regression Models

Table of contents
1. [Preparing the data](#dropping-non-essential-columns-to-linear-regression)
2. [Linear Regression](#linear-regression)
   - [All features](#1a-using-all-features)
   - [Genres](#1b-using-only-genres)
   - [Studios](#1c-using-only-studios)
   - [Media Type](#1d-only-mediatype)
   - [Genres + Media Type](#1e-using-genres-and-mediatype)
3. [Cross Validation](#cross-validation)
4. [Ridge Regression](#2-ridge-regression)
5. [Lasso Regression](#3-lasso-regression)
6. [Best regression model](#analysis-of-all-regression-models)

---

### Essential Libraries

Let us begin by importing the essential Python Libraries.

> NumPy : Library for Numeric Computations in Python  
> Pandas : Library for Data Acquisition and Preparation  
> Matplotlib : Low-level library for Data Visualization  
> Seaborn : Higher-level library for Data Visualization  

In [6]:
# Basic Libraries
import json
import statistics
import math

from collections import defaultdict

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt  # we only need pyplot

sb.set()  # set the default Seaborn style for graphics


## Advantages and disadvantages of various linear regression models

The following is from [https://statisticsbyjim.com/regression/choosing-regression-analysis/](https://statisticsbyjim.com/regression/choosing-regression-analysis/)

`Ordinary Least Squares (OLS)`
- Sensitivity to both outliers and multicollinearity
- Prone to overfitting

`Ridge regression`
- allows you to analyze data even when severe multicollinearity is present
- helps prevent overfitting
- reduces the large, problematic variance that multicollinearity causes by introducing a slight bias in the estimates
- trades away much of the variance in exchange for a little bias, which produces more useful coefficient estimates when multicollinearity is present.

`Lasso regression` (least absolute shrinkage and selection operator)
- performs variable selection that aims to increase prediction accuracy by identifying a simpler model
- It is similar to Ridge regression but with variable selection

`Partial least squares` (PLS) regression
- is useful when you have very few observations compared to the number of independent variables or when your independent variables are highly correlated. 
- PLS decreases the independent variables down to a smaller number of uncorrelated components, similar to Principal Components Analysis. 
- Then, the procedure performs linear regression on these components rather than the original data. 
- PLS emphasizes developing predictive models and is not used for screening variables. 
- Unlike OLS, you can include multiple continuous dependent variables. 
- PLS uses the correlation structure to identify smaller effects and model multivariate patterns in the dependent variables.

## Importing the data

In [7]:
anime_df = pd.read_csv('dataset/anime_cleaned_2.csv')
print("Number of animes:", len(anime_df))
anime_df.head(1)

Number of animes: 8661


,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,...,broadcast_day_of_the_week,broadcast_start_time,statistics_watching,statistics_completed,statistics_on_hold,statistics_dropped,statistics_plan_to_watch,statistics_num_list_users,positive_viewership_fraction,negative_viewership_fraction
0,95,Turn A Gundam,1999-04-09,2000-04-14,"It is the Correct Century, two millennia after...",7.71,1049,2892,40743,13338,...,friday,17:00,2735.0,16661.0,2538.0,1597.0,17292.0,40823.0,0.8987,0.1013


## Dropping non-essential columns to linear regression

In [8]:
anime_df.drop([
    'title', 'start_date', 'end_date', 'id', 'synopsis', 'rank', 'popularity',
    'num_list_users', 'num_scoring_users', 'broadcast_day_of_the_week',
    'broadcast_start_time', 'statistics_watching', 'statistics_completed',
    'statistics_on_hold', 'statistics_dropped', 'statistics_plan_to_watch',
    'statistics_num_list_users', 'positive_viewership_fraction', 'negative_viewership_fraction'
],
              axis=1,
              inplace=True)


In [9]:
anime_df.head(1)

,mean,nsfw,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,start_season_year,start_season_season
0,7.71,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",50,original,1445,pg_13,"[{'id': 14, 'name': 'Sunrise'}, {'id': 1260, '...",1999.0,spring


## Unravel `genres` and `studios` from one column to multiple columns

In [10]:
from ipynb.fs.full.helpers import json_genres, json_studios

anime_df = json_genres(anime_df)  # convert genres column to json
anime_df = json_studios(anime_df)  # convert studios column to json

c:\Programming Projects (Not Synced)\dsai\SC1015-Project\Anime\helpers.ipynb:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "\n",
c:\Programming Projects (Not Synced)\dsai\SC1015-Project\Anime\helpers.ipynb:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "outputs": [],


In [11]:
def unravel_genre_or_studio(row):
    res = pd.Series(dtype=str)
    for elem in row:
        res = res.append(pd.Series([elem['name']]))
    res.reset_index(inplace=True, drop=True)
    return res


genres_expanded = anime_df['genres'].apply(
    lambda row: unravel_genre_or_studio(row))
genres_expanded

,0,1,2,3,4,5,6,7,8,9,10
0,Action,Adventure,Drama,Mecha,Military,Romance,Sci-Fi,Space,NaN,NaN,NaN
1,Action,Drama,Military,Sci-Fi,Space,NaN,NaN,NaN,NaN,NaN,NaN
2,Adventure,Comedy,Fantasy,Kids,Sci-Fi,Shounen,NaN,NaN,NaN,NaN,NaN
3,Action,Adventure,Comedy,Drama,Fantasy,Shounen,Super Power,NaN,NaN,NaN,NaN
4,Adventure,Comedy,Kids,Sci-Fi,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8656,Music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8657,Comedy,Slice of Life,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8658,Adventure,Comedy,Demons,Fantasy,Historical,Supernatural,NaN,NaN,NaN,NaN,NaN
8659,Music,Supernatural,Vampire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
genres_expanded = genres_expanded.fillna('NA')

In [13]:
studios_expanded = anime_df['studios'].apply(
    lambda row: unravel_genre_or_studio(row))
studios_expanded = studios_expanded.fillna('NA')
studios_expanded

,0,1,2,3,4,5,6,7,8,9
0,Sunrise,Nakamura Production,NA,NA,NA,NA,NA,NA,NA,NA
1,Artland,Magic Bus,NA,NA,NA,NA,NA,NA,NA,NA
2,Shin-Ei Animation,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,Toei Animation,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,Toei Animation,NA,NA,NA,NA,NA,NA,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...
8656,Doga Kobo,NA,NA,NA,NA,NA,NA,NA,NA,NA
8657,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8658,Sunrise,NA,NA,NA,NA,NA,NA,NA,NA,NA
8659,A-1 Pictures,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [14]:
anime_expanded_df = anime_df.copy()
for index, row in genres_expanded.iterrows():
    for i in genres_expanded.columns:
        anime_expanded_df.loc[index, f"genre-{i}"] = genres_expanded.iloc[index, i]
for index, row in studios_expanded.iterrows():
    for i in studios_expanded.columns:
        anime_expanded_df.loc[index, f"studio-{i}"] = studios_expanded.iloc[index, i]


In [15]:
anime_expanded_df.head(2)

,mean,nsfw,media_type,status,genres,num_episodes,source,average_episode_duration,rating,studios,...,studio-0,studio-1,studio-2,studio-3,studio-4,studio-5,studio-6,studio-7,studio-8,studio-9
0,7.71,white,tv,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 2, 'name'...",50,original,1445,pg_13,"[{'id': 14, 'name': 'Sunrise'}, {'id': 1260, '...",...,Sunrise,Nakamura Production,NA,NA,NA,NA,NA,NA,NA,NA
1,8.07,white,ova,finished_airing,"[{'id': 1, 'name': 'Action'}, {'id': 8, 'name'...",28,novel,1560,r,"[{'id': 8, 'name': 'Artland'}, {'id': 207, 'na...",...,Artland,Magic Bus,NA,NA,NA,NA,NA,NA,NA,NA


### Encoding nominal (unordered) categorical variables using `OneHotEncoding`

Our dataset contains a lot of categorical variables such as:
- media_type
- source
- rating
- start_season_season
- start_season_year
- status
- nsfw
- genres
- studios

In [16]:
# Import the encoder from sklearn
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

# OneHotEncoding of categorical predictors (not the response)
cat_variables = [
    'media_type', 'source', 'rating', 'start_season_season',
    'start_season_year', 'status', 'nsfw'
] + [f"genre-{i}" for i in genres_expanded.columns] + [f"studio-{i}" for i in studios_expanded.columns]
anime_cat = anime_expanded_df[cat_variables]

ohe.fit(anime_cat)
anime_cat_ohe = pd.DataFrame(ohe.transform(anime_cat).toarray(),
                             columns=ohe.get_feature_names(anime_cat.columns))

# Check the encoded variables
anime_cat_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8661 entries, 0 to 8660
Columns: 917 entries, media_type_movie to studio-9_Steve N' Steven
dtypes: float64(917)
memory usage: 60.6 MB


In [17]:
num_variable = []
for i in anime_df:
    if i not in cat_variables:
        num_variable.append(i)
num_variable

['mean', 'genres', 'num_episodes', 'average_episode_duration', 'studios']

In [18]:
# Combining Numeric features with the OHE Categorical features
animeData_num = anime_df[num_variable]
animeData_ohe = pd.concat([animeData_num, anime_cat_ohe],
                          sort=False,
                          axis=1).reindex(index=animeData_num.index)

# Check the final dataframe
animeData_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8661 entries, 0 to 8660
Columns: 922 entries, mean to studio-9_Steve N' Steven
dtypes: float64(918), int64(2), object(2)
memory usage: 60.9+ MB


## 1) Linear Regression

### 1a) Using `all features`

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Extract Response and Predictors
y = pd.DataFrame(animeData_ohe['mean'])
X = pd.DataFrame(animeData_ohe.drop(['mean', 'genres', 'studios'], axis=1))

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [38]:
from sklearn.metrics import mean_squared_error

# fit model
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# predict
y_train_pred = linreg.predict(X_train)

# Goodness of Fit for Train Data
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:",
      mean_squared_error(y_train, y_train_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_train, y_train_pred)))
print()

# Accuracy for Test Data
y_test_pred = linreg.predict(X_test)
print("Accuracy of Model        \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_test, y_test_pred)))
print()

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.48726496686721166
Mean Squared Error (MSE) 	: 0.3199368048186707
Root Mean Squared Error (RMSE) 	: 0.5656295650146576

Accuracy of Model        	Test Dataset
Explained Variance (R^2) 	: -3.859731382790666e+19
Mean Squared Error (MSE) 	: 2.5039385100255654e+19
Root Mean Squared Error (RMSE) 	: 5003936960.060114



#### 1ai) Analysis of using `all features`
High `train` accuracy but low `test` accuracy signifies that there is *overfitting*

Now, we will try to reduce the number of variables used.

### 1b) Using only `genres`

In [49]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Extract Response and Predictors
y = pd.DataFrame(animeData_ohe['mean'])
X = pd.DataFrame(animeData_ohe[[col for col in animeData_ohe if 'genre-' in col]])

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# fit model
linreg_genre = LinearRegression()
linreg_genre.fit(X_train, y_train)

# predict
y_train_pred = linreg_genre.predict(X_train)

# Goodness of Fit for Train Data
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg_genre.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:",
      mean_squared_error(y_train, y_train_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_train, y_train_pred)))
print()

# Accuracy for Test Data
y_test_pred = linreg_genre.predict(X_test)
print("Accuracy of Model        \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg_genre.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_test, y_test_pred)))
print()

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.4593199232178331
Mean Squared Error (MSE) 	: 0.3397409563442853
Root Mean Squared Error (RMSE) 	: 0.5828730190567113

Accuracy of Model        	Test Dataset
Explained Variance (R^2) 	: 0.40293469072122323
Mean Squared Error (MSE) 	: 0.38131173624976966
Root Mean Squared Error (RMSE) 	: 0.6175044422915269



#### 1bi) Analysis of using `genres`
Explained variance of less than 0.5 signifies that just using genres is not good enough.

We need to explore other variables as well.

### 1c) using only studios

In [114]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Extract Response and Predictors
y = pd.DataFrame(animeData_ohe['mean'])
X = pd.DataFrame(
    animeData_ohe[[col for col in animeData_ohe if 'studio-' in col]])

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# fit model
linreg_studio = LinearRegression()
linreg_studio.fit(X_train, y_train)

# predict
y_train_pred = linreg_studio.predict(X_train)

# Goodness of Fit for Train Data
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg_studio.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:",
      mean_squared_error(y_train, y_train_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_train, y_train_pred)))
print()

# Accuracy for Test Data
y_test_pred = linreg_studio.predict(X_test)
print("Accuracy of Model        \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg_studio.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_test, y_test_pred)))
print()

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.40595259683061913
Mean Squared Error (MSE) 	: 0.37205481849904504
Root Mean Squared Error (RMSE) 	: 0.6099629648585602

Accuracy of Model        	Test Dataset
Explained Variance (R^2) 	: -2.2595199916753316e+25
Mean Squared Error (MSE) 	: 1.4522369345266013e+25
Root Mean Squared Error (RMSE) 	: 3810822659907.702



#### 1ci) Analysis of using `studios`
Explained variance of less than 0.5 signifies that just using studio is not good enough.

We need to explore other variables as well.

### 1d) Only media_type

In [121]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Extract Response and Predictors
y = pd.DataFrame(animeData_ohe['mean'])
X = pd.DataFrame(animeData_ohe[[col for col in animeData_ohe if 'media_type_' in col]])

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# fit model
linreg_media_type = LinearRegression()
linreg_media_type.fit(X_train, y_train)

# predict
y_train_pred = linreg_media_type.predict(X_train)

# Goodness of Fit for Train Data
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg_media_type.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:",
      mean_squared_error(y_train, y_train_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_train, y_train_pred)))
print()

# Accuracy for Test Data
y_test_pred = linreg_media_type.predict(X_test)
print("Accuracy of Model        \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg_media_type.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_test, y_test_pred)))
print()

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.09587828765434481
Mean Squared Error (MSE) 	: 0.5743520966401788
Root Mean Squared Error (RMSE) 	: 0.7578602091680093

Accuracy of Model        	Test Dataset
Explained Variance (R^2) 	: 0.10502703067547903
Mean Squared Error (MSE) 	: 0.5570186004063312
Root Mean Squared Error (RMSE) 	: 0.7463367875204405



#### 1di) Analysis of using `media_type`
Explained variance of 0.10 signifies that just using `media_type` is a poor indicator

We need to explore other variables as well.

### 1e) Using genres and media_type

In [136]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Extract Response and Predictors
y = pd.DataFrame(animeData_ohe['mean'])
X = pd.DataFrame(animeData_ohe[[col for col in animeData_ohe if 'genre-' in col or 'media_type_' in col]])

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# fit model
linreg_genre_media_type = LinearRegression()
linreg_genre_media_type.fit(X_train, y_train)

# predict
y_train_pred = linreg_genre_media_type.predict(X_train)

# Goodness of Fit for Train Data
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg_genre_media_type.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:",
      mean_squared_error(y_train, y_train_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_train, y_train_pred)))
print()

# Accuracy for Test Data
y_test_pred = linreg_genre_media_type.predict(X_test)
print("Accuracy of Model        \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg_genre_media_type.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print("Root Mean Squared Error (RMSE) \t:",
      np.sqrt(mean_squared_error(y_test, y_test_pred)))
print()

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.47636826415060396
Mean Squared Error (MSE) 	: 0.328582784468714
Root Mean Squared Error (RMSE) 	: 0.5732214096391672

Accuracy of Model        	Test Dataset
Explained Variance (R^2) 	: 0.4493060684999124
Mean Squared Error (MSE) 	: 0.35266130380436694
Root Mean Squared Error (RMSE) 	: 0.5938529311238322



#### 1ei) Analysis of using `media_type`
Explained variance of 0.5 signifies that just using `genres` and `media_type` is not enough

We need to explore other variables as well.

### Cross validation function

In [178]:
# get mean of explained variance [r2] with 5 fold cross-validation
def get_cv_r2(model, X, y):
    scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    print(f'R2 mean: {np.mean(scores)}')

### 2) Ridge Regression
Ridge regression can help reduce the complexity of our model to prevent overfitting

In [179]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV

# Extract Response and Predictors
y = pd.DataFrame(animeData_ohe['mean'])
X = pd.DataFrame(animeData_ohe.drop(['mean', 'genres', 'studios'], axis=1))

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Train model
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)

# get cross val scores
get_cv_r2(ridge, X_train, y_train)

# determine best alpha
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)
grid = GridSearchCV(estimator=ridge,
                    param_grid=param_grid,
                    scoring='r2',
                    verbose=1,
                    n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
print('Best r2:\t ', grid_result.best_score_)
print('Best alpha:\t ', grid_result.best_params_['alpha'])

R2 mean: 0.6251537328722826
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best r2:	  0.6251537328722827
Best alpha:	  1


In [180]:
# Calculate accuracy for the most optimal model
optimal_ridge = Ridge(alpha=grid_result.best_params_['alpha'])
optimal_ridge.fit(X_train, y_train)
print("Accuracy (R2)")
print("Train:\t", optimal_ridge.score(X_train, y_train))
print("Test:\t", optimal_ridge.score(X_test, y_test))

Accuracy (R2)
Train:	 0.7131523970496216
Test:	 0.6297478461177066


#### Analysis of Ridge Regression

`R2` of train and test is greater than that of linear regression and is above `0.5`

Hence, ridge regression is a better model than linear regression

### 3) Lasso Regression

As we have `300+` features, we can use lasso regression to select those that are the most important.

In [181]:
from sklearn.linear_model import Lasso

# Train model
lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)

# get cross val scores
get_cv_r2(lasso, X_train, y_train)

# determine best alpha
alpha = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
param_grid = dict(alpha=alpha)
grid = GridSearchCV(estimator=lasso,
                    param_grid=param_grid,
                    scoring='r2',
                    verbose=1,
                    n_jobs=-1)
grid_result = grid.fit(X_train, y_train)
print('Best r2:\t ', grid_result.best_score_)
print('Best alpha:\t ', grid_result.best_params_['alpha'])

R2 mean: 0.0694195091523361
Fitting 5 folds for each of 7 candidates, totalling 35 fits
Best r2:	  0.5903881883512543
Best alpha:	  0.001


In [182]:
# Calculate accuracy for the most optimal model
optimal_lasso = Lasso(alpha=grid_result.best_params_['alpha'])
optimal_lasso.fit(X_train, y_train)
print("Accuracy (R2)")
print("Train:\t", optimal_lasso.score(X_train, y_train))
print("Test:\t", optimal_lasso.score(X_test, y_test))

Accuracy (R2)
Train:	 0.6169736289136505
Test:	 0.5830801903157012


#### Analysis of Lasso Regression

`R2` of train and test is greater than that of linear regression and is above `0.5` but is lower than that of ridge regression

Hence, lasso regression is a better model than linear regression but is a poorer model than ridge regression

### Best regression model

Ranking of regression models:
1. Ridge regression (`~0.7`)
2. Lasso regression (`~0.6`)
3. Linear regression (`~0.4`)
